### mediapipe 두손 정적 단어

* feature 30개 ( 왼손 마디의 각도 15개, 오른손 마디의 각도 15개)

In [2]:
import cv2 # 웹캠 제어 및 ML 사용 
import mediapipe as mp # 손 인식을 할 것
import numpy as np
import pandas as pd
import time
from PIL import ImageFont, ImageDraw, Image


import warnings
warnings.filterwarnings('ignore')

In [73]:
def mediapipe(model):
    max_num_hands = 2 # 손은 최대 1개만 인식

    # MediaPipe hands model
    mp_hands = mp.solutions.hands # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티1
    mp_drawing = mp.solutions.drawing_utils # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티2

     # 손가락 detection 모듈을 초기화
    hands = mp_hands.Hands(  
        max_num_hands=max_num_hands, # 최대 몇 개의 손을 인식? 
        min_detection_confidence=0.5, # 0.5로 해두는 게 좋다!  
        min_tracking_confidence=0.5)  


    cap = cv2.VideoCapture(0) 
    data = np.zeros(30).reshape(1,30)
    start_time = time.time()
    pre_text, sentence, delay_time = '', '', 1.5
    while cap.isOpened(): # 웹캠에서 한 프레임씩 이미지를 읽어옴
        ret, img = cap.read()
        if not ret:
            return

        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        result = hands.process(img)

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        angle_arr = []
        # 각도를 인식하고 제스처를 인식하는 부분 
        if result.multi_hand_landmarks is not None: # 만약 손을 인식하면 
            for res in result.multi_hand_landmarks: 
                joint = np.zeros((21, 3)) # joint == 랜드마크에서 빨간 점, joint는 21개가 있고 x,y,z 좌표니까 21,3
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z] # 각 joint마다 x,y,z 좌표 저장

                # Compute angles between joints joint마다 각도 계산 
                # **공식문서 들어가보면 각 joint 번호의 인덱스가 나옴**
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
                v = v2 - v1 # [20,3]관절벡터 
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화(크기 1 벡터) = v / 벡터의 크기

                # Get angle using arcos of dot product **내적 후 arcos으로 각도를 구해줌** 
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                angle = np.degrees(angle) # Convert radian to degree
                angle_arr.extend(angle)

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS) # 손에 랜드마크를 그려줌 
                
            if len(angle_arr) == 15:
                angle_arr = np.concatenate([angle_arr, np.zeros(15)])
                
            if len(angle_arr) > 30:
                continue
            angle_arr = np.array(angle_arr).reshape(1, 30)
            
            text = str(model.predict(angle_arr)[0])
            
            if text != pre_text:
                start_time = time.time()
                pre_text = text
                
            else:
                if time.time() - start_time > delay_time:
                    sentence += text
                    start_time = time.time()
            
            
            font = ImageFont.truetype("fonts/gulim.ttc", 20)
            img = Image.fromarray(img)
            draw = ImageDraw.Draw(img)
            draw.text((30,50), text, font=font, fill=(0,0,255))
            img = np.array(img)
           
            data = np.append(data, angle_arr , axis=0)
        
        font = ImageFont.truetype("fonts/gulim.ttc", 20)
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text((200,0), sentence, font=font, fill=(0,0,255))
        img = np.array(img)
    
        cv2.imshow('show', img)
        if cv2.waitKey(1) == ord('q'):
            break

    cv2.destroyAllWindows()
    cap.release()
    
    return 

In [7]:
def save():
    max_num_hands = 2 # 손은 최대 1개만 인식

    # MediaPipe hands model
    mp_hands = mp.solutions.hands # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티1
    mp_drawing = mp.solutions.drawing_utils # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티2

     # 손가락 detection 모듈을 초기화
    hands = mp_hands.Hands(  
        max_num_hands=max_num_hands, # 최대 몇 개의 손을 인식? 
        min_detection_confidence=0.5, # 0.5로 해두는 게 좋다!  
        min_tracking_confidence=0.5)  


    cap = cv2.VideoCapture(0) 

    data = np.zeros(30).reshape(1,30)

    while cap.isOpened(): # 웹캠에서 한 프레임씩 이미지를 읽어옴
        ret, img = cap.read()
        if not ret:
            return

        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        result = hands.process(img)

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        angle_arr = []
        # 각도를 인식하고 제스처를 인식하는 부분 
        if result.multi_hand_landmarks is not None: # 만약 손을 인식하면 
            for res in result.multi_hand_landmarks: 
                joint = np.zeros((21, 3)) # joint == 랜드마크에서 빨간 점, joint는 21개가 있고 x,y,z 좌표니까 21,3
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z] # 각 joint마다 x,y,z 좌표 저장

                # Compute angles between joints joint마다 각도 계산 
                # **공식문서 들어가보면 각 joint 번호의 인덱스가 나옴**
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
                v = v2 - v1 # [20,3]관절벡터 
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화(크기 1 벡터) = v / 벡터의 크기

                # Get angle using arcos of dot product **내적 후 arcos으로 각도를 구해줌** 
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                angle = np.degrees(angle) # Convert radian to degree
                angle_arr.extend(angle)

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS) # 손에 랜드마크를 그려줌 
                
            if len(angle_arr) == 15:
                angle_arr = np.concatenate([angle_arr, np.zeros(15)]) 
            if len(angle_arr) > 30:
                continue
            angle_arr = np.array(angle_arr).reshape(1, 30)
            data = np.append(data, angle_arr , axis=0)

        cv2.imshow('show', img)
        if cv2.waitKey(1) == ord('q'):
            break

    cv2.destroyAllWindows()
    cap.release()
    
    return data

In [35]:
d = save()

In [36]:
d = pd.DataFrame(d)
d['label'] = '나는'
d

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
1,18.098181,15.430419,9.364005,43.525588,6.617929,7.987697,33.491051,10.353205,1.185566,32.149258,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
2,15.708549,2.976240,15.921370,44.672581,5.738898,4.549972,33.188086,4.093285,2.391934,20.195657,...,25.525495,1.137648,4.671806,15.868731,0.672473,3.657429,27.879637,10.48171,4.73266,나는
3,15.718893,1.788963,12.106222,53.535680,2.234917,4.252674,36.847218,0.983958,2.945870,20.904425,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
4,15.792852,2.414430,16.572045,50.302411,3.552417,6.411055,34.038294,3.334981,1.769047,21.231786,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,14.546117,2.398487,15.631336,44.193962,3.476385,3.360150,31.184312,3.498751,1.167252,18.035411,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
63,14.181392,2.675894,15.640250,43.984990,3.413335,3.699231,30.979556,3.858078,1.145557,18.313092,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
64,13.885300,2.461976,16.617416,44.275548,3.806191,3.634976,30.804217,3.600551,0.633494,18.254175,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는
65,14.156076,2.404823,16.665148,44.315794,3.978757,3.433730,31.031663,3.487946,0.909632,17.828052,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,나는


In [57]:
df = pd.read_csv('./hi.csv')
df = pd.concat([df, d])

In [60]:
df.head(30)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
1,20.265298,8.657827,20.467216,44.562597,11.721517,4.377248,32.575988,9.220238,4.142376,16.742578,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
2,16.428906,8.429271,13.987454,46.207242,7.711435,6.321428,32.528064,6.414908,4.071506,17.461699,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
3,13.762877,7.522706,12.577969,46.514305,7.893335,6.797767,34.428732,6.031740,4.374249,19.741196,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
4,15.378705,7.772545,16.896561,44.405791,7.281996,6.335256,31.491636,4.081484,2.399667,16.303101,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
5,16.856779,8.356963,17.476169,54.509912,5.331008,6.011762,40.408080,7.428495,1.582640,22.016239,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,감사합니다
6,12.320014,11.038689,20.603025,18.138559,14.124682,17.819637,12.858269,10.653798,12.215177,6.895984,...,38.067613,5.969857,4.099399,19.837849,8.821349,1.398170,28.236105,10.726509,1.798334,감사합니다
7,22.947901,8.709407,13.697481,27.381572,3.871540,15.596403,24.376139,8.480186,12.303337,28.109807,...,37.836076,6.995575,1.035335,19.610578,9.444065,2.480456,26.502076,9.819951,2.281425,감사합니다
8,16.426969,8.544282,15.836530,28.239650,3.924258,12.831559,23.770839,4.833547,10.361983,27.126715,...,37.791526,6.914333,3.546492,20.073599,10.257942,0.567678,29.275596,11.222888,1.368889,감사합니다
9,24.258790,6.174187,13.241908,22.906104,4.351283,14.905161,18.514810,9.230415,11.936803,24.090218,...,39.555224,8.310896,3.688761,20.787523,10.632244,0.486055,26.165048,10.910012,1.310393,감사합니다


In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

In [62]:
target ='label'
x = df.drop(target, axis=1)
y = df[target]

In [63]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

In [64]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [65]:
y_pred = knn.predict(x_test)
accuracy_score(y_test, y_pred)

0.967741935483871

In [72]:
mediapipe(knn)